In [2]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

In [3]:
#block 1 is closer to wall and block 2 is further away from the wall
#collisions with wall and blocks are described as of two diferent types and represented using variable collison_type throughout the code
#velocity of block can be of any value , number of collisions will still give pi
#positions are mesured from left edge of block
decimal_places = 2                              #number of decimal places till which the user wants to pridict pi
x1_initial = 4                                 #position of block 1 
x2_initial = 10                                #position of block 2
v1_initial = 0                                  #velocity of block 1
v2_initial = -200                                 #velocity of block 2
m1 = 1                                          #mass of block 1
m2 = (100)**decimal_places                      #mass of block 2
l1 = 1                                          #side length of block 1           
l2 = decimal_places*l1                          #side length of block 2

In [4]:
#defining functions
def new_velocity(v1_initial,v2_initial,collision_type): #computes velocity of blocks after collision
    #if block hits wall
    v1_wall = -v1_initial
    v2_wall = v2_initial
    
    #if two blocks hit each other
    v1_block = ((m1 - m2)*v1_initial + 2*m2*v2_initial)/(m1 + m2)
    v2_block = ((m2 - m1)*v2_initial + 2*m1*v1_initial)/(m1 + m2)
    
    final_velocity = {'wall':[v1_wall, v2_wall],'block':[v1_block, v2_block]}
    return final_velocity[collision_type][0] , final_velocity[collision_type][1]

def compute_time(x1,x2,collision_type):        #computes time beweenpresent position and next collision
    t_wall = (-x1)/v1
    t_block = (l1 + x1-x2)/(v2-v1)
    time={'wall':t_wall , 'block':t_block}
    return time[collision_type] 

In [5]:
#Computing position of blocks during collisions
x1 = np.array([x1_initial])              #Stores position of first block      
x2 = np.array([x2_initial])              #Stores position of second block
v1 = v1_initial                          #Velocity of first block
v2 = v2_initial                          #Velocity of second block
collision_index = []                     #Stores index of x1 and x2 at which collision occurs
collision_type={0:'block' , 1:'wall'}    #Assigns each type of collision a binary number
no_collisions=0                          #Number of collisions
while True:
    if v1>0 and v2>v1:                   #No more collisions will happen after this condition is reached
         break
    collision = collision_type[no_collisions % 2]              #Collision that is going to take place next        
    time_interval = compute_time(x1[-1],x2[-1],collision)      #computes time interval untill next collision
   
    n = int(np.ceil(time_interval*10000))        #Number of steps for euler's inegeration     
    dt = time_interval/n                          #time step
    for k in range(0,n):                         #Euler's integeration to compute positions till next collsion 
        x1 = np.append(x1, x1[-1]+v1*dt)
        x2 = np.append(x2, x2[-1]+v2*dt)

    collision_index.append(len(x2)-1)            #stores index of x1 and x2 at which collision took place
    v1,v2 = new_velocity(v1,v2,collision)        #Computes velocities after collision
    no_collisions += 1                    
    
print(f'Number of Collisions: {no_collisions}')

#Result
print(f'Predicted value of pi is: {no_collisions*(10**(-decimal_places))}')

Number of Collisions: 314
Predicted value of pi is: 3.14


C:\Users\ashis\AppData\Local\Temp\ipykernel_18436\1506056439.py:15: RuntimeWarning: divide by zero encountered in long_scalars
  t_wall = (-x1)/v1


In [6]:
#Animation
%matplotlib qt      
# %matplotlib qt: Magic command to display matplotlib objects in seperate window [won't work in idle]

fig, ax = plt.subplots(figsize=(10,10))                  #Creating figure and axes
block_1 = plt.Rectangle((x1[0],0), l1, l1,  fc='red')        #Creating first block       
block_2 = plt.Rectangle((x2[0],0), l2, l2,  fc='blue')       #Creating second block
ax.add_patch(block_1)                    #Adds rectangle object to axes
ax.add_patch(block_2)

#Setting limit for x and y axes
ax.set_xlim(0,10)
ax.set_ylim(0,l2+1)

no_collisions = 0          #reinitializing number of collisions

def update(frame):        
    global no_collisions  
    if frame in collision_index:        #Checks if collision takes place at present frame   
        no_collisions = no_collisions + 1      
    
    #Updating position of blocks for current frame
    block_1.set_xy((x1[frame],0))       
    block_2.set_xy((x2[frame], 0))
    
    plt.title(f'Number of Collision:{no_collisions}')       #Adds no. of collisions as title to plot 
    return block_1, block_2

ax.set_aspect('equal')                 #Sets aspect ratio of plot to equal
animate = FuncAnimation(fig, update, frames=len(x1), interval=1)
plt.show()